Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

train_df.head(10)

TEXT_LABELS = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [3]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:16000]
y_test=label[16000:]

## 2) textcuboid 분류

In [4]:
textcuboid=np.load('./1-Channel textcuboid_ag(elmo).npy')

In [5]:
textcuboid_test=np.load('./1-Channel textcuboid_test_ag(elmo).npy')

In [6]:
tmp = [[x,y] for x, y in zip(textcuboid, y_train)]
import random
random.shuffle(tmp)
textcuboid = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [7]:
import keras
from tensorflow.keras import layers

In [26]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [27]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model


input_layer = Input(shape=(46, 256))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)

x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(4, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)



In [28]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [29]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 46, 256)]         0         
                                                                 
 conv1d_2 (Conv1D)           (None, 46, 1024)          263168    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 1024)             0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 256)               262400    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 4)                 1028      
                                                           

In [30]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train1=y_train[1000:]
y_val=y_train[:1000]

In [31]:
history = model.fit(x_train, y_train1,callbacks=callbacks, epochs=20,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/20
59/59 [==============================] - 1s 9ms/step - loss: 0.6436 - accuracy: 0.7905 - val_loss: 0.3059 - val_accuracy: 0.9010 - lr: 0.0010
Epoch 2/20
59/59 [==============================] - 0s 6ms/step - loss: 0.3158 - accuracy: 0.8981 - val_loss: 0.2830 - val_accuracy: 0.9100 - lr: 0.0010
Epoch 3/20
59/59 [==============================] - 0s 7ms/step - loss: 0.2715 - accuracy: 0.9110 - val_loss: 0.2718 - val_accuracy: 0.9120 - lr: 0.0010
Epoch 4/20
59/59 [==============================] - 0s 6ms/step - loss: 0.2361 - accuracy: 0.9195 - val_loss: 0.2641 - val_accuracy: 0.9140 - lr: 0.0010
Epoch 5/20
59/59 [==============================] - 0s 6ms/step - loss: 0.2088 - accuracy: 0.9282 - val_loss: 0.2632 - val_accuracy: 0.9130 - lr: 0.0010
Epoch 6/20
59/59 [==============================] - 0s 6ms/step - loss: 0.1776 - accuracy: 0.9400 - val_loss: 0.2821 - val_accuracy: 0.9120 - lr: 0.0010
Epoch 7/20
59/59 [==============================] - 0s 6ms/step - loss: 0.1535 - a

In [32]:
y_test=np.array(y_test)

In [33]:
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 2ms/step - loss: 0.2751 - accuracy: 0.9185


[0.2751215994358063, 0.9185000061988831]

In [34]:
from keras.models import load_model
model = load_model('best_model.h5')
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 1ms/step - loss: 0.2327 - accuracy: 0.9185


[0.23272055387496948, 0.9185000061988831]

In [54]:
np.average([92.0,91.6,91.8])

91.8